In [ ]:
!pip install transformers pandas scikit-learn reportlab

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score
from transformers import pipeline
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from google.colab import files

# Set up the Hugging Face summarization pipeline
summarizer = pipeline("summarization")


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [ ]:
def upload_excel_file():
    uploaded = files.upload()
    for fn in uploaded.keys():
        return pd.read_excel(fn)

# Upload norms and college information Excel files
print("Upload the Excel file containing the norms:")
norms_df = upload_excel_file()

print("Upload the Excel file containing the college information:")
college_df = upload_excel_file()


Upload the Excel file containing the norms:


Saving questions_and_answers (1).xlsx to questions_and_answers (1).xlsx
Upload the Excel file containing the college information:


Saving college_data.xlsx to college_data.xlsx


In [ ]:
import pandas as pd

def compare_excels_ml(norms_df, college_df):
    comparison_results = {}

    # Define the column index or name to compare (e.g., column 2)
    column_to_compare = norms_df.columns[1]  # Assuming column 2 is the one you want to compare

    for column in norms_df.columns:
        if column == column_to_compare:
            # Convert columns to numeric, coercing errors to NaN
            norms_col = pd.to_numeric(norms_df[column], errors='coerce')
            college_col = pd.to_numeric(college_df[column], errors='coerce')

            # Create a list to store comparison results for each row
            column_comparison = []

            for index, (norm_val, college_val) in enumerate(zip(norms_col, college_col)):
                entity = norms_df.iloc[index, 0]  # Get the entity from column 1

                if pd.isna(norm_val) or pd.isna(college_val):
                    column_comparison.append(f"{entity}: Data missing or invalid.")
                elif college_val < norm_val:
                    column_comparison.append(f"{entity}:  is less than the norm.")
                elif college_val == norm_val:
                    column_comparison.append(f"{entity}:  matches the norm.")
                else:
                    column_comparison.append(f"{entity}: is greater than the norm.")

            # Store the results for this column
            comparison_results[column] = column_comparison

    return comparison_results


In [ ]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch

def generate_pdf_report_ml(comparison_results):
    pdf_file = "/content/Compliance_Report_ML.pdf"
    doc = SimpleDocTemplate(pdf_file, pagesize=letter)

    elements = []

    # Add a title with a larger font size and bold style
    title_style = ParagraphStyle(
        name="TitleStyle",
        fontSize=24,
        fontName="Helvetica-Bold",
        alignment=1,  # Center alignment
        spaceAfter=24,
        textColor=colors.darkblue
    )
    title = Paragraph("Compliance Report", title_style)
    elements.append(title)

    # Iterate through comparison results and create boxes
    for field, result in comparison_results.items():
        for index, value in enumerate(result):
            # Assuming 'less than the norm' indicates non-compliance
            if 'less than the norm' in value:
                box_background = colors.lightcoral  # Highlight with a red background
            else:
                box_background = colors.lightgrey  # Default background

            # Extract entity from the result string
            entity = value.split(':')[0].strip()

            # Add a box around each entity
            box_data = [
                [Paragraph("<b>Entity:</b>", getSampleStyleSheet()['Heading4']), Paragraph(entity, getSampleStyleSheet()['Normal'])],
                [Paragraph("<b>Result:</b>", getSampleStyleSheet()['Heading4']), Paragraph(value, getSampleStyleSheet()['Normal'])]
            ]
            box_table = Table(box_data, colWidths=[1.5*inch, 5.5*inch])
            box_table.setStyle(TableStyle([
                ('BACKGROUND', (0, 0), (-1, 0), colors.lightblue),
                ('BACKGROUND', (0, 1), (-1, -1), box_background),
                ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
                ('ALIGN', (0, 0), (-1, -1), 'LEFT'),
                ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                ('FONTSIZE', (0, 0), (-1, 0), 12),
                ('FONTSIZE', (0, 1), (-1, -1), 10),
                ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                ('BOX', (0, 0), (-1, -1), 2, colors.blue),
                ('VALIGN', (0, 0), (-1, -1), 'TOP')
            ]))
            elements.append(box_table)
            elements.append(Spacer(1, 12))  # Add space between boxes

    # Add a summary using NLP
    summary_input = " ".join([f"{field}: {', '.join(result)}" for field, result in comparison_results.items()])
    summary_input = summary_input[:512]  # Adjust 512 based on the model's limit
    summary = summarizer(summary_input, max_length=100, min_length=30, do_sample=False)
    summary_text = summary[0]['summary_text']

    # Add the summary to the PDF elements with a custom style
    summary_style = ParagraphStyle(
        name="SummaryStyle",
        fontSize=12,
        fontName="Helvetica-Oblique",
        spaceBefore=24,
        textColor=colors.darkgreen
    )
    summary_paragraph = Paragraph("<b>Summary:</b><br/>" + summary_text, summary_style)
    elements.append(summary_paragraph)
    doc.build(elements)
    print(f"PDF report generated: {pdf_file}")

# Generate the report
comparison_results = compare_excels_ml(norms_df, college_df)
generate_pdf_report_ml(comparison_results)


PDF report generated: /content/Compliance_Report_ML.pdf
